# Step 1 — Object Segmentation with SAM3

In [ ]:
!git clone https://github.com/RizwanMunawar/sam3-inference
%cd sam3-inference
!pip install -e .
!pip install decord

In [ ]:
import cv2
import numpy as np
from PIL import Image
from sam3 import build_sam3_image_model
from sam3.model.sam3_image_processor import Sam3Processor
from sam3.visualize.utils import draw_box_and_masks
import os

In [ ]:
model_path = "/content/sam3.pt"

print("[INFO] Loading SAM3 model...")
processor = Sam3Processor(build_sam3_image_model(checkpoint_path=model_path))

In [ ]:
# solid back
from PIL import Image
import numpy as np
import cv2
import os

prompts = ["bear"]
output_images_black = "/content/bear_black_bg"
output_masks = "/content/mask_path"

os.makedirs(output_images_black, exist_ok=True)
os.makedirs(output_masks, exist_ok=True)

for img_name in sorted(os.listdir("/content/bear/images")):
    if not img_name.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    img_path = f"/content/bear/images/{img_name}"

    image_pil = Image.open(img_path).convert("RGB")
    image_cv  = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)

    state = processor.set_image(image_pil)
    final_mask = np.zeros((image_pil.height, image_pil.width), dtype=np.uint8)

    for prompt in prompts:
        results = processor.set_text_prompt(state=state, prompt=prompt)
        masks  = results["masks"]

        for mask in masks:
            binary_mask = mask.cpu().numpy().astype(np.uint8).squeeze()
            final_mask[binary_mask == 1] = 255

    mask_name = f"{img_name}.png"
    cv2.imwrite(os.path.join(output_masks, mask_name), final_mask)


    segmented_image_black = np.zeros_like(image_cv, dtype=np.uint8)
    for c in range(3):
        segmented_image_black[:, :, c] = np.where(final_mask == 255, image_cv[:, :, c], 0)

    cv2.imwrite(os.path.join(output_images_black, img_name), segmented_image_black)

    print(f"Processed {img_name} | Mask: {mask_name} | BG: Black")

# Step 2 — Sparse COLMAP with Masks

In [ ]:
!apt-get update
!apt-get install -y colmap libgl1-mesa-glx libglib2.0-0

In [ ]:
!mkdir -p /content/scenebear_mask

In [ ]:
!colmap feature_extractor \
    --database_path /content/scenebear_mask/database.db \
    --image_path /content/bear_black_bg \
    --ImageReader.mask_path /content//mask_path \
    --SiftExtraction.use_gpu 0


In [ ]:
!colmap exhaustive_matcher \
    --database_path /content/scenebear_mask/database.db \
    --SiftMatching.use_gpu 0

In [ ]:
!mkdir -p /content/scenebear_mask/sparse

!colmap mapper \
    --database_path /content/scenebear_mask/database.db \
    --image_path /content/bear_black_bg \
    --output_path /content/scenebear_mask/sparse \
    --Mapper.multiple_models 0

In [ ]:
!colmap image_undistorter \
    --image_path /content/bear_black_bg \
    --input_path /content/scenebear_mask/sparse/0 \
    --output_path /content/scenebear_mask/distorted \
    --output_type COLMAP

# Step 3 — Convert Background to White (Required for 3DGS)

In [ ]:
import cv2
import os
import numpy as np
from glob import glob
from tqdm import tqdm

input_path = "/content/scenebear_mask/distorted/images"
output_path = input_path

images = glob(os.path.join(input_path, "*.png")) + glob(os.path.join(input_path, "*.jpg"))

for img_path in tqdm(images):
    img = cv2.imread(img_path)
    if img is None:
        continue

    lower_black = np.array([0, 0, 0])
    upper_black = np.array([30, 30, 30])
    mask = cv2.inRange(img, lower_black, upper_black)

    img[mask > 0] = [255, 255, 255]

    filename = os.path.basename(img_path)
    cv2.imwrite(os.path.join(output_path, filename), img)

In [ ]:
!rm -rf /content/scenebear_mask/sparse && mkdir -p /content/scenebear_mask/sparse

!colmap mapper \
    --database_path /content/scenebear_mask/database.db \
    --image_path /content/scenebear_mask/distorted/images \
    --output_path /content/scenebear_mask/sparse \
    --Mapper.multiple_models 0

In [ ]:
# Step 4 — Dense COLMAP with PyCOLMAP (CUDA)

In [ ]:
!pip install pycolmap-cuda12==3.13.0

In [ ]:
import pycolmap
from pathlib import Path

mvs_path = "/content/scenebear_mask/distorted"

try:
    pycolmap.patch_match_stereo(mvs_path)
    output_ply = f"{mvs_path}/fused.ply"
    pycolmap.stereo_fusion(output_ply, mvs_path)

except Exception as e:
    print({e})

In [ ]:
import subprocess

def generate_poisson_mesh(input_ply, output_mesh):
    cmd = [
    "colmap", "poisson_mesher",
    "--input_path", input_ply,
    "--output_path", output_mesh,
    "--PoissonMeshing.trim", "5",
    "--PoissonMeshing.depth", "10"
    ]
    subprocess.run(cmd, check=True)


generate_poisson_mesh(
f"{mvs_path}/fused.ply",
f"{mvs_path}/mesh_colmap.ply"
)

# Step 6 — Train Vanilla 3D Gaussian Splatting (3DGS)

In [ ]:
!git clone https://github.com/graphdeco-inria/gaussian-splatting.git
%cd gaussian-splatting
!git submodule update --init --recursive
%cd submodules/simple-knn
!pip install .
%cd ../diff-gaussian-rasterization
!pip install .
!pip install -q plyfile


In [ ]:
%cd /content/gaussian-splatting

# distorted/sparse/0
!python train.py \
  -s /content/scenebear_mask/distorted \
  --iterations 5000 \
  -r 1 \
  --white_background \
  --densify_grad_threshold 0.0002 \
  --opacity_reset_interval 700


# Step 7 — SuGaR: Gaussians → Mesh

In [ ]:
%cd /content/

In [ ]:
!git clone --recursive https://github.com/Anttwo/SuGaR.git
%cd SuGaR
!pip install plyfile==0.8.1 trimesh open3d nvdiffrast
!pip install "git+https://github.com/facebookresearch/pytorch3d.git@stable"

In [ ]:
import json

json_path = "/cameras.json"  # fix path for 3dgs vanilla output
with open(json_path, "r") as f:
    data = json.load(f)

for frame in data:
    if frame["img_name"].endswith(".JPG"):
        frame["img_name"] = frame["img_name"].replace(".JPG", ".jpg")

with open(json_path, "w") as f:
    json.dump(data, f, indent=2)

print("JSON corrigido")


In [ ]:
frames = data if isinstance(data, list) else data.get("frames", [])

for frame in frames:
    name = frame.get("img_name", "")
    frame["img_name"] = os.path.splitext(name)[0]

with open(json_path, "w") as f:
    json.dump(data, f, indent=2)


In [ ]:
import cv2
import os

IMG_DIR = "/content/scenebear_mask/distorted/images"
TARGET_W = 984
TARGET_H = 728

for f in os.listdir(IMG_DIR):
    if not f.lower().endswith((".jpg", ".png")):
        continue

    path = os.path.join(IMG_DIR, f)
    img = cv2.imread(path)

    if img is None:
        continue

    resized = cv2.resize(
        img,
        (TARGET_W, TARGET_H),
        interpolation=cv2.INTER_AREA
    )

    cv2.imwrite(path, resized)


In [ ]:
!python train_full_pipeline.py \
  -s /content/scenebear_mask/distorted/ \
  -r dn_consistency \
  --high_poly True \
  --export_obj True \
  --gs_output_dir /content/gaussian-splatting/output \
  --white_background True